# Imports & Loading Data

In [2]:
import numpy as np
import pandas as pd
#import pycountry_convert as pc

pd.set_option('display.max_rows', 1000)
# dti = pd.to_datetime(['1/1/2018', np.datetime64('2018-01-01'),datetime.datetime(2018, 1, 1)])
covid_data = pd.read_csv('./alldays_data.csv', parse_dates= ['Last_Update'],
date_parser = pd.to_datetime) # adjust later code for parsing date here

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)


Exception: File `'./data_fetching_part01.ipynb'` not found.

## Fixing String columns 
* some errands 
* strip whitespaces 
* fillna province with country 
* fillna combined_key

In [ ]:
# some errands
some_corrections = {'Mainland China': 'China', 'US': 'USA', 'Korea, South': 'South Korea',
                    'Taiwan*' : 'Taiwan', 'Congo (Kinshasa)' : "Democratic Republic of the Congo",
                    "Cote d'Ivoire": "Côte d'Ivoire", "Reunion": "Réunion", 'UK': 'United Kingdom',
                    'Congo (Brazzaville)': 'Republic of the Congo', 'Bahamas, The': 'Bahamas',
                    'Gambia, The': 'Gambia', 'The Gambia': 'Gambia', 'West Bank and Gaza': 'Palestine',
                    'Burma': "Myanmar", 'Timor-Leste': "East Timor", 'Republic of Korea': 'South Korea',
                    'Iran (Islamic Republic of)': 'Iran', 'Viet Nam': 'Vietnam', 'Hong Kong SAR': 'Hong Kong',
                    'Russian Federation': 'Russia', 'occupied Palestinian territory': 'Palestine',
                     'The Bahamas': 'Bahamas', 'Macao SAR': 'Macau', 'Republic of Ireland': 'Ireland'}

covid_data['Country'] = covid_data['Country'].replace(some_corrections)

covid_data.loc[covid_data['Province']=='None', 'Province'] = np.nan
# striping leading and trailing whitespaces from string variables
covid_data[['Country', 'Province', 'Combined_Key']] = \
covid_data[['Country', 'Province', 'Combined_Key']].apply(lambda col: col.str.strip(), axis=0)
# fillna provoince with name of country
covid_data['Province'] = covid_data.apply(lambda x: x['Country'] if pd.isna(x['Province']) else x['Province'], axis = 1)
def fillna_combined_key(row):
    if pd.isna(row['Combined_Key']):
         # fill with country if no province-level country, with country & province otherwise
        row['Combined_Key'] = row['Country'] if row['Country'] == row['Province'] else row['Country'] + ', ' + row['Province']
    return row

covid_data = covid_data.apply(lambda row: fillna_combined_key(row), axis=1)

In [ ]:
# Found that JHU, source, used a more disaggregated reporting starting from 1st Feb in some countries e.g. USA, Canada, Australia. So decided to avoid the problem it might do to the way I got new cases from accumulated cases, by dropping all before 1st Feb except China which accounted for majority of cases at the time.

covid_data = covid_data[(covid_data['Last_Update'] >= "2020-02-01 00:00:00") | covid_data['Country'].isin(['China', 'Macau', 'Hong Kong'])]

## Arranging data
* Arranging Columns
* two dataframes (Us vs Other World)


In [ ]:
covid_data.columns = covid_data.columns.str.lower()
covid_data.rename(columns = {'province': 'state', 'last_update': 'date', 'combined_key': 'location'}, inplace=True)
covid_data = covid_data[['country', 'state', 'date', 'confirmed', 'deaths', 'recovered', 'active', 'location', 'fips']]

#### World data

In [ ]:
df_world = covid_data[covid_data['country'] != 'USA'].copy()
df_world = df_world.drop(columns = 'fips') # only relevant for USA
df_world.head()

,country,state,date,confirmed,deaths,recovered,active,location
0,China,Anhui,2020-01-22 17:00:00,1.0,NaN,NaN,NaN,"China, Anhui"
1,China,Beijing,2020-01-22 17:00:00,14.0,NaN,NaN,NaN,"China, Beijing"
2,China,Chongqing,2020-01-22 17:00:00,6.0,NaN,NaN,NaN,"China, Chongqing"
3,China,Fujian,2020-01-22 17:00:00,1.0,NaN,NaN,NaN,"China, Fujian"
4,China,Gansu,2020-01-22 17:00:00,NaN,NaN,NaN,NaN,"China, Gansu"


In [ ]:
df_world = df_world.drop_duplicates(subset=['country', 'state', 'date'], keep='last') # per_day cases: last report each day if more than one
# numeric columns
num_cols = ['confirmed', 'deaths', 'recovered', 'active']
df_world.loc[:, num_cols] = df_world.loc[:, num_cols].fillna(0)

# # per day cases (confirmed, deaths) Note: original data is accumulated over time as far as I know
# I guess there should be a better solution than looping on each group
df_world.sort_values(by=['country', 'state', 'date'], inplace=True) # I think sort here is important 
df_world.reset_index(drop = True, inplace=True)
grouped = df_world.groupby(['country', 'state'])
all_data = []
for _, group in grouped:
    for col in num_cols[:-1]:
        new_col = 'daily_' + col
        group[new_col] = group[col].diff(1)
        group.loc[group.index[0], new_col] = group.loc[group.index[0], col] # very first value the accumulated and daily col is same        
    all_data.append(group)
        
df_world = pd.concat(all_data, sort=False, ignore_index=True)
df_world.sort_values(by=['country', 'state', 'date'], inplace=True)

#### Per Country Cases

In [ ]:
# per country cases
#1st grouping to get country or country with state data (i.e. agg daily data and last of accumulated)
per_country_cases = df_world.groupby(['country', 'state'], as_index=False).\
agg({'confirmed': 'last', 'deaths': 'last', 'recovered': 'last', 'active': 'median',
     'daily_confirmed':'sum', 'daily_deaths': 'sum', 'daily_recovered': 'sum'}) 
#2nd grouping to get country level from states (won't harm no-state level data)
per_country_cases = per_country_cases.groupby('country', as_index=False).sum()
per_country_cases = per_country_cases[['country', 'confirmed', 'deaths', 'recovered', 'active']]

In [ ]:
per_country_cases.head()

,country,confirmed,deaths,recovered,active
0,Afghanistan,58730.0,2572.0,52392.0,5273.0
1,Albania,130409.0,2372.0,105016.0,5295.0
2,Algeria,120736.0,3198.0,84167.0,13555.5
3,Andorra,13024.0,124.0,12458.0,413.0
4,Angola,25492.0,577.0,23092.0,1125.0


#### daily cases

In [ ]:
df_world['date'] = df_world['date'].dt.normalize() # drop unnecessary time part
df_daily = df_world.groupby(['country', 'state', 'date'], as_index=False).sum()
df_daily = df_daily[['country', 'state', 'date', 'daily_confirmed', 'daily_deaths', 'daily_recovered', 'active']]
df_daily.rename(columns = {col: col[6:] for col in ['daily_confirmed', 'daily_deaths', 'daily_recovered']}, inplace=True)
# dropping negative numbers from daily cases (I guess errors in reporting)
df_daily.loc[df_daily['confirmed'] < 0, 'confirmed'] = np.nan
df_daily.loc[df_daily['deaths'] < 0, 'deaths'] = np.nan
df_daily.loc[df_daily['recovered'] < 0, 'recovered'] = np.nan
df_daily.dropna(subset=['confirmed', 'deaths', 'recovered'], inplace=True)

In [ ]:
df_daily.head()

,country,state,date,confirmed,deaths,recovered,active
0,Afghanistan,Afghanistan,2020-02-24,1.0,0.0,0.0,0.0
1,Afghanistan,Afghanistan,2020-03-08,3.0,0.0,0.0,0.0
2,Afghanistan,Afghanistan,2020-03-10,1.0,0.0,0.0,0.0
3,Afghanistan,Afghanistan,2020-03-11,2.0,0.0,0.0,0.0
4,Afghanistan,Afghanistan,2020-03-14,4.0,0.0,0.0,0.0


#### Adding continent column
* Better after aggregation

In [ ]:
# Continent_code to Continent_names
continents = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU' : 'Europe',
    'na' : 'Others'
}

def country_to_continent_code(country):
    try:
        return pc.country_alpha2_to_continent_code(pc.country_name_to_country_alpha2(country))
    except:
        return "na"

# insert continent column
df_daily.insert(0, "continent", df_daily['country'].apply(lambda x: continents[country_to_continent_code(x)]))
per_country_cases.insert(0, "continent", per_country_cases['country'].apply(lambda x: continents[country_to_continent_code(x)]))

### per country cases: from worldometer

In [ ]:
df_other = pd.read_csv('./world_worldometer.csv')
columns = {'Continent': 'continent', 'Country Other':'country', 'TotalCases': 'confirmed', 'TotalDeaths': 'deaths',
          'TotalRecovered': 'recovered', 'ActiveCases': 'active', 'TotalTests': 'tests', 'Population':'population'}
df_other = df_other.rename(columns = columns)
df_other = df_other[[col for _, col in columns.items()]]
df_other = df_other[1:]
df_other.sort_values('country', inplace=True)
df_other.head()

,continent,country,confirmed,deaths,recovered,active,tests,population
103,Asia,Afghanistan,59021,2592.0,52489,3940,395439.0,39638567.0
83,Europe,Albania,130537,2378.0,105728,22431,622711.0,2875230.0
84,Africa,Algeria,120922,3207.0,84299,33416,230861.0,44493653.0
137,Europe,Andorra,13060,124.0,12491,445,193595.0,77366.0
120,Africa,Angola,25609,579.0,23092,1938,455499.0,33691594.0


#### USA
* stopped working on it, it seems some data are cumulative, others are new cases, not sure how to handle
* using worldometer data as a cross-section for latest USA data

In [ ]:
df_us = pd.read_csv('./usa_worldometer.csv')
columns = {'USAState':'state', 'TotalCases': 'confirmed', 'TotalDeaths': 'deaths',
          'ActiveCases': 'active', 'TotalTests': 'tests'}
df_us = df_us.rename(columns = columns)
df_us.insert(0, 'country', 'USA')
df_us.insert(0, 'continent', 'North America')
df_us.insert(0, 'date', pd.Timestamp.today().normalize())
df_us = df_us[['date', 'continent', 'country', 'state', 'confirmed', 'deaths', 'active', 'tests']]
df_us = df_us[1:]
df_us.head() # can be used for state-level analysis

,date,continent,country,state,confirmed,deaths,active,tests
1,2021-04-26,North America,USA,California,3732256,61479,1687173.0,59095717
2,2021-04-26,North America,USA,Texas,2877774,50176,91177.0,27678766
3,2021-04-26,North America,USA,Florida,2208584,34861,414122.0,27309151
4,2021-04-26,North America,USA,New York,2077439,52242,585139.0,50361096
5,2021-04-26,North America,USA,Illinois,1321033,24139,97476.0,22269555


In [ ]:
us_row = df_us[['confirmed', 'deaths', 'active']].sum()
us_row['country'] = 'USA'
us_row['continent'] = 'North America'

per_country_cases = per_country_cases.append(us_row, ignore_index=True, sort=False)
per_country_cases.sort_values('country', inplace=True)
df_daily = df_daily.append(df_us, sort=False, ignore_index=True)
df_daily.sort_values(['country', 'state', 'date'], inplace=True)

# Saving Cleaned Data to csv

In [ ]:
import os
if not os.path.exists('./cleaned_data'):
    os.mkdir('cleaned_data')
per_country_cases.to_csv('./cleaned_data/per_country_aggregate.csv', index=False)
df_other.to_csv('./cleaned_data/allcountries_worldometer.csv', index=False)
print('Successfully saved: ./cleaned_data/allcountries_worldometer.csv')
print('Successfully saved: ./cleaned_data/per_country_aggregate.csv')
df_daily.to_csv('./cleaned_data/daily_disagg.csv', index=False)
print('Successfully saved: ./cleaned_data/daily_disagg.csv')
df_us.to_csv('./cleaned_data/usa_states.csv', index=False)
print('Successfully saved: ./cleaned_data/usa_states.csv')

Successfully saved: ./cleaned_data/allcountries_worldometer.csv
Successfully saved: ./cleaned_data/per_country_aggregate.csv
Successfully saved: ./cleaned_data/daily_disagg.csv
Successfully saved: ./cleaned_data/usa_states.csv
